---
## Torch → JAX model initialisation
---
This notebook shows how to take an existing Torch MACE model, convert it
to the JAX implementation using the same helpers that power the
`mace_torch2jax` CLI, and then run both versions on the same example batch.

In [1]:
import json
from pathlib import Path

import ase
import jax
import jax.numpy as jnp
import numpy as np
import torch

jax.config.update('jax_enable_x64', True)

from mace.data.atomic_data import AtomicData
from mace.data.utils import KeySpecification, config_from_atoms
from mace.tools import build_default_arg_parser, check_args, torch_geometric
from mace.tools.model_script_utils import configure_model as configure_model_torch
from mace.tools.multihead_tools import AtomicNumberTable, prepare_default_head
from mace.tools.scripts_utils import extract_config_mace_model
from mace.tools.torch_geometric.batch import Batch

from mace_jax.cli.mace_torch2jax import convert_model


/home/philipp/Source/mace-jax/mace-jax/.venv/lib/python3.13/site-packages/e3nn/o3/_wigner.py:10: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  _Jd, _W3j_flat, _W3j_indices = torch.load(os.path.join(os.path.dirname(__file__), 'constants.pt'))


---
### Build a Torch reference model
---

In [2]:
def load_statistics(path: Path = Path('statistics.json')):
    data = json.loads(path.read_text())
    stats = dict(data)
    stats['atomic_numbers'] = AtomicNumberTable(stats['atomic_numbers'])
    stats['atomic_energies'] = [
        stats['atomic_energies'][str(z)] for z in stats['atomic_numbers'].zs
    ]
    return stats


def build_example_args():
    arguments = [
        '--name', 'MACE_large_density',
        '--interaction_first', 'RealAgnosticDensityInteractionBlock',
        '--interaction', 'RealAgnosticDensityResidualInteractionBlock',
        '--num_channels', '128',
        '--max_L', '2',
        '--max_ell', '3',
        '--num_interactions', '3',
        '--correlation', '3',
        '--num_radial_basis', '8',
        '--MLP_irreps', '16x0e',
        '--distance_transform', 'Agnesi',
        '--pair_repulsion',
        '--only_cueq', 'True',
    ]
    parser = build_default_arg_parser()
    args = parser.parse_args(arguments)
    args, _ = check_args(args)
    return args


def build_torch_model(args, statistics):
    args.mean = statistics['mean']
    args.std = statistics['std']
    args.compute_energy = True
    args.compute_dipole = False
    args.key_specification = KeySpecification.from_defaults()
    args.heads = prepare_default_head(args)

    torch.set_default_dtype(torch.float32)
    model, _ = configure_model_torch(
        args,
        None,
        statistics['atomic_energies'],
        z_table=statistics['atomic_numbers'],
        heads=args.heads,
        model_foundation=None,
    )
    model.eval()
    return model


In [3]:
statistics = load_statistics(Path('statistics.json'))
torch_args = build_example_args()
torch_model = build_torch_model(torch_args, statistics)


/home/philipp/Source/mace-jax/mace-jax/.venv/lib/python3.13/site-packages/cuequivariance_torch/primitives/segmented_polynomial.py:170: UserWarning: cuequivariance_ops_torch is not available. Falling back to naive implementation.
  warnings.warn(
/home/philipp/Source/mace-jax/mace-jax/.venv/lib/python3.13/site-packages/cuequivariance_torch/primitives/segmented_polynomial.py:170: UserWarning: cuequivariance_ops_torch is not available. Falling back to naive implementation.
  warnings.warn(
/home/philipp/Source/mace-jax/mace-jax/.venv/lib/python3.13/site-packages/cuequivariance_torch/primitives/segmented_polynomial.py:170: UserWarning: cuequivariance_ops_torch is not available. Falling back to naive implementation.
  warnings.warn(
/home/philipp/Source/mace-jax/mace-jax/.venv/lib/python3.13/site-packages/cuequivariance_torch/primitives/segmented_polynomial.py:170: UserWarning: cuequivariance_ops_torch is not available. Falling back to naive implementation.
  warnings.warn(
/home/philipp/So

---
### Convert Torch model to JAX
---

In [4]:
config = extract_config_mace_model(torch_model)
config['torch_model_class'] = torch_model.__class__.__name__
jax_model, variables, template_batch = convert_model(torch_model, config)


---
### Prepare an evaluation batch
---

In [5]:
def batch_to_jax(batch: Batch) -> dict[str, jnp.ndarray]:
    converted = {}
    for key in batch.keys:
        value = batch[key]
        if isinstance(value, torch.Tensor):
            converted[key] = jnp.asarray(value.detach().cpu().numpy())
        else:
            converted[key] = value
    return converted

# Example SrTiO3 structure as before
a0 = 3.905
atoms = ase.Atoms(
    symbols=['Sr', 'Ti', 'O', 'O', 'O'],
    positions=a0 * np.array([
        [0.0, 0.0, 0.0],
        [0.5, 0.5, 0.5],
        [0.5, 0.5, 0.0],
        [0.5, 0.0, 0.5],
        [0.0, 0.5, 0.5],
    ]),
    cell=a0 * np.identity(3),
    pbc=[True, True, True],
)
config_atoms = config_from_atoms(atoms)
config_atoms.pbc = [bool(x) for x in config_atoms.pbc]
atomic_data = AtomicData.from_config(
    config_atoms,
    z_table=statistics['atomic_numbers'],
    cutoff=torch_args.r_max,
)
batch_torch = torch_geometric.batch.Batch.from_data_list([atomic_data])
batch_jax = batch_to_jax(batch_torch)


---
### Compare Torch and JAX outputs
---

In [6]:
jax_out = jax_model.apply(variables, batch_jax, compute_stress=True)
torch_out = torch_model(batch_torch, compute_stress=True)

print('JAX energy:', jax_out['energy'])
print('Torch energy:', torch_out['energy'])
print('JAX forces:', jax_out['forces'])
print('Torch forces:', torch_out['forces'])
print('JAX stress:', jax_out['stress'])
print('Torch stress:', torch_out['stress'])


JAX energy: [-34.5185879]
Torch energy: tensor([-34.5215], grad_fn=<AddBackward0>)
JAX forces: [[-6.0674664e-17  2.0857339e-17  1.8302688e-17]
 [-2.9368326e-17  7.2396733e-18 -1.9569849e-17]
 [ 2.5905656e-17  3.7523560e-17  2.1616281e-18]
 [ 5.4847077e-17 -9.0598644e-17  3.9525945e-17]
 [ 7.8906496e-18  2.0451807e-17 -4.2128031e-17]]
Torch forces: tensor([[ 2.4011e-08,  1.6369e-08, -9.3041e-09],
        [-3.5361e-09, -4.4811e-09,  1.4261e-08],
        [-1.3324e-08,  1.2011e-08, -2.8012e-10],
        [ 2.1444e-08, -2.5695e-08,  2.3992e-09],
        [-2.8860e-08,  3.0858e-09, -9.0422e-09]])
JAX stress: [[[-1.9014148e-03  1.6342133e-11  7.2764225e-12]
  [ 1.6342133e-11 -1.9014148e-03  4.1749989e-12]
  [ 7.2764225e-12  4.1749989e-12 -1.9014152e-03]]]
Torch stress: tensor([[[-1.8856e-03, -1.0691e-09,  3.9137e-09],
         [-1.0691e-09, -1.8856e-03,  1.1355e-09],
         [ 3.9137e-09,  1.1355e-09, -1.8856e-03]]])
